In [1]:
import numpy 
import pandas 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from PIL import Image
import glob
import zipfile

In [ ]:
train_files=glob.glob('C:/Users/000110888/OneDrive - CSULB/Desktop/PetsImages/train/*.jpg')
train_labels=[i.strip('C:/Users/000110888/OneDrive - CSULB/Desktop/PetsImages/train/')[1:4] for i in train_files]
train_df=pandas.DataFrame({'filename': train_files, 'class': train_labels})
fig,axs=plt.subplots(2, 2, figsize=(10,10))
axs=axs.ravel()
for i in range(0,4):
    idx = random.choice(train_df.index)
    axs[i].imshow(Image.open(train_df['filename'][idx]))
    axs[i].set_title(train_df['class'][idx])

In [3]:
train_datagen=ImageDataGenerator(rotation_range=5,rescale=1./255, horizontal_flip=True, shear_range=0.2,
zoom_range=0.2, validation_split=0.2)

img_height,img_width=224, 224
batch_size=64

train_generator=train_datagen.flow_from_dataframe(train_df, target_size=(img_height, img_width),
batch_size=batch_size, class_mode='categorical', subset='training')

validation_generator=train_datagen.flow_from_dataframe(train_df, target_size=(img_height, img_width),
batch_size=batch_size, class_mode='categorical', subset='validation')

Found 72 validated image filenames belonging to 2 classes.
Found 18 validated image filenames belonging to 2 classes.


In [ ]:
#training CNN model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model=Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2 ,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', metrics=['accuracy'])

history=model.fit(train_generator, epochs=55, validation_data=validation_generator)

In [ ]:
test_files=glob.glob('C:/Users/000110888/OneDrive - CSULB/Desktop/PetsImages/test/*.jpg')
test_df=pandas.DataFrame({'filename': test_files})
test_gen=ImageDataGenerator(rescale=1./255)
test_generator=test_gen.flow_from_dataframe(test_df, x_col='filename', y_col=None,
class_mode=None, target_size=(img_height,img_width), batch_size=batch_size)

def visualize_predictions(test_generator, model):
    plt.figure(figsize=(12,12))
    for i in range(0, 35):
        plt.subplot(7, 5, i+1)
        for X_batch in test_generator:
            prediction=model.predict(X_batch)[0]
            image=X_batch[0]
            plt.imshow(image)
            plt.title('cat' if numpy.argmax(prediction)==0 else 'dog')
            break
    plt.tight_layout()
    plt.show()
    
visualize_predictions(test_generator, model)